<a href="https://colab.research.google.com/github/ummehanikonok16/ImageNet/blob/main/imageNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = r'/content/drive/MyDrive/AAV/train_val_photos-20240207T143515Z-001/train_val_photos'

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
    transforms.ToTensor(),
])

In [ ]:
from torchvision.datasets import ImageFolder
dataset = ImageFolder(root=data_dir, transform=transform)

In [ ]:
from torch.utils.data import DataLoader, random_split
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
    transforms.ToTensor(),
])

In [ ]:
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
NUM_CLASSES = len(os.listdir('/content/drive/MyDrive/AAV/train_val_photos-20240207T143515Z-001/train_val_photos'))

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/AAV/train_val_photos-20240207T143515Z-001/train_val_photos',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 1937 images belonging to 3 classes.


In [ ]:
validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/AAV/train_val_photos-20240207T143515Z-001/train_val_photos',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 1937 images belonging to 3 classes.


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)


Epoch 1/10
60/60 [==============================] - 473s 8s/step - loss: 1.0709 - accuracy: 0.4866 - val_loss: 0.9939 - val_accuracy: 0.5354
Epoch 2/10
60/60 [==============================] - 472s 8s/step - loss: 1.0008 - accuracy: 0.5265 - val_loss: 0.8942 - val_accuracy: 0.5203
Epoch 3/10
60/60 [==============================] - 504s 8s/step - loss: 0.9892 - accuracy: 0.5459 - val_loss: 0.8721 - val_accuracy: 0.6146
Epoch 4/10
60/60 [==============================] - 504s 8s/step - loss: 0.9489 - accuracy: 0.5465 - val_loss: 0.9433 - val_accuracy: 0.5073
Epoch 5/10
60/60 [==============================] - 470s 8s/step - loss: 0.9345 - accuracy: 0.5780 - val_loss: 0.9365 - val_accuracy: 0.5604
Epoch 6/10
60/60 [==============================] - 504s 8s/step - loss: 0.8963 - accuracy: 0.5995 - val_loss: 0.7999 - val_accuracy: 0.6625
Epoch 7/10
60/60 [==============================] - 504s 8s/step - loss: 0.9002 - accuracy: 0.6021 - val_loss: 0.7970 - val_accuracy: 0.6812
Epoch 8/10
60

In [ ]:
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/AAV/train_val_photos-20240207T143515Z-001/train_val_photos',
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

Found 1937 images belonging to 3 classes.


In [ ]:
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

print(classification_report(true_classes, predicted_classes, target_names=class_labels))

1937/1937 [==============================] - 235s 121ms/step
              precision    recall  f1-score   support

      Double       0.35      0.81      0.49       376
       Empty       0.74      0.71      0.72       616
      Single       0.88      0.46      0.60       945

    accuracy                           0.61      1937
   macro avg       0.66      0.66      0.61      1937
weighted avg       0.73      0.61      0.62      1937

